In [29]:
import h5py
import pandas
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, log_loss, mean_absolute_percentage_error
from scipy.stats import randint, poisson, uniform

def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        filename = name.split('/')[1]
        return pandas.DataFrame(f[filename][:], dtype=np.float64)

train = load_data('data/train')
test  = load_data('data/test')

In [30]:
print (f'Shape of training data set: {train.shape}')
print (f'Shape of test data set: {test.shape}')

Shape of training data set: (162500, 166)
Shape of test data set: (160651, 164)


In [31]:
all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]

In [32]:
X = train[all_variables]
y = train['Truth']

e_index = y.loc[y==1].index

X_e = X.iloc[e_index]
y_e = train['p_truth_E'].iloc[e_index]


print (f'Shape of X: {X_e.shape}')
print (f'Shape of y: {y_e.shape}')

Shape of X: (121495, 160)
Shape of y: (121495,)


In [33]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [34]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'logloss'

In [35]:
#evallist = [(dtest, 'eval'), (dtrain, 'train')]
#num_round = 10
#bst = xgb.train(param, dtrain, num_round, evallist)
#print(np.shape(X))
#print(np.shape(y))

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X_e, y_e, test_size=0.20, random_state=7)

In [37]:
model = XGBRegressor(max_depth = 10, eta = 0.3, eval_metric = "mape",n_estimators=1000)
model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=10)


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[0]	validation_0-mape:0.70015
[1]	validation_0-mape:0.49680
[2]	validation_0-mape:0.35755
[3]	validation_0-mape:0.26210
[4]	validation_0-mape:0.19687
[5]	validation_0-mape:0.15251
[6]	validation_0-mape:0.12209
[7]	validation_0-mape:0.10194
[8]	validation_0-mape:0.08907
[9]	validation_0-mape:0.08094
[10]	validation_0-mape:0.07630
[11]	validation_0-mape:0.07337
[12]	validation_0-mape:0.07170
[13]	validation_0-mape:0.07079
[14]	validation_0-mape:0.07026
[15]	validation_0-mape:0.06999
[16]	validation_0-mape:0.06987
[17]	validation_0-mape:0.06963
[18]	validation_0-mape:0.06952
[19]	validation_0-mape:0.06940
[20]	validation_0-mape:0.06941
[21]	validation_0-mape:0.06941
[22]	validation_0-mape:0.06943
[23]	validation_0-mape:0.06951
[24]	validation_0-mape:0.06954
[25]	validation_0-mape:0.06958
[26]	validation_0-mape:0.06956
[27]	validation_0-mape:0.06959
[28]	validation_0-mape:0.06962
[29]	validation_0-mape:0.06964


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             eta=0.3, eval_metric='mape', gamma=0, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [38]:
y_pred = model.predict(X_val,iteration_range=(0,model.best_iteration))
print(y_pred)


[145187.77 236826.81 147377.61 ... 147944.47 265332.1  130187.36]


In [39]:
#kfold = KFold(n_splits=10, shuffle=True)
#kf_cv_scores = cross_val_score(model, X, y, cv=kfold )
#print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [40]:
import shap

In [41]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [42]:
#shap.plots.bar(shap_values, max_display=50)


In [43]:
feature_names = X_train.columns
rf_resultX = pandas.DataFrame(shap_values, columns = feature_names)
vals = np.abs(rf_resultX.values).mean(0)
shap_importance = pandas.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
shap_importance.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)
imp_var = shap_importance['col_name'].iloc[:12].tolist()
print(imp_var)


['p_eCluster', 'p_eAccCluster', 'p_rawECluster', 'p_ecore', 'p_nCells_Lr1_HiG', 'p_deltaPhi2', 'p_deltaEta2', 'p_nTracks', 'p_E3x5_Lr1', 'p_wtots1', 'p_widths1', 'p_EptRatio']


In [44]:
X_20 = X_e[imp_var]
y = train['Truth']

print (f'Shape of X: {X_20.shape}')
print (f'Shape of y: {y_e.shape}')

Shape of X: (121495, 12)
Shape of y: (121495,)


In [45]:
X_train, X_val, y_train, y_val = train_test_split(X_20, y_e, test_size=0.10, random_state=7)

In [46]:
model2 = XGBRegressor(max_depth = 15, eta = 0.4, eval_metric = 'mape',n_estimators=1000)
model2.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=10)

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[0]	validation_0-mape:0.60433
[1]	validation_0-mape:0.37834
[2]	validation_0-mape:0.24712
[3]	validation_0-mape:0.17135
[4]	validation_0-mape:0.12807
[5]	validation_0-mape:0.10464
[6]	validation_0-mape:0.09295
[7]	validation_0-mape:0.08814
[8]	validation_0-mape:0.08592
[9]	validation_0-mape:0.08514
[10]	validation_0-mape:0.08500
[11]	validation_0-mape:0.08503
[12]	validation_0-mape:0.08501
[13]	validation_0-mape:0.08504
[14]	validation_0-mape:0.08506
[15]	validation_0-mape:0.08489
[16]	validation_0-mape:0.08491
[17]	validation_0-mape:0.08495
[18]	validation_0-mape:0.08506
[19]	validation_0-mape:0.08517
[20]	validation_0-mape:0.08520
[21]	validation_0-mape:0.08519
[22]	validation_0-mape:0.08527
[23]	validation_0-mape:0.08533
[24]	validation_0-mape:0.08542
[25]	validation_0-mape:0.08559


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             eta=0.4, eval_metric='mape', gamma=0, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.400000006, max_delta_step=0, max_depth=15,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [47]:
dist_param = {'n_estimators':[1000],
              'learning_rate':uniform(0.01,0.5),
              'max_depth':randint(5,15),
              'eval_metric':[mean_absolute_percentage_error]}

fit_param = {'early_stopping_rounds':5,
             'eval_set':[(X_val,y_val)],
             'verbose':0}



In [48]:
# run randomized search
n_iter_search = 20
RandomSearch = RandomizedSearchCV(model2, 
                                  param_distributions=dist_param, 
                                  n_iter=n_iter_search, 
                                  cv=5, 
                                  return_train_score=True,
                                  random_state=7,
                                  scoring='neg_mean_absolute_error',
                                  n_jobs=1,
                                  verbose=10
                                 )

In [49]:
RandomSearch.fit(X_train,y_train,**fit_param);

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5; 1/20] START eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.04815414468697859, max_depth=14, n_estimators=1000


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[CV 1/5; 1/20] END eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.04815414468697859, max_depth=14, n_estimators=1000;, score=(train=-3730.599, test=-6531.210) total time=  38.6s
[CV 2/5; 1/20] START eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.04815414468697859, max_depth=14, n_estimators=1000


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[CV 2/5; 1/20] END eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.04815414468697859, max_depth=14, n_estimators=1000;, score=(train=-3692.758, test=-6606.459) total time=  40.8s
[CV 3/5; 1/20] START eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.04815414468697859, max_depth=14, n_estimators=1000


...

[CV 3/5; 20/20] END eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.1843781878273576, max_depth=14, n_estimators=1000;, score=(train=-4088.304, test=-6844.519) total time=  12.0s
[CV 4/5; 20/20] START eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.1843781878273576, max_depth=14, n_estimators=1000


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[CV 4/5; 20/20] END eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.1843781878273576, max_depth=14, n_estimators=1000;, score=(train=-4035.381, test=-6950.435) total time=  11.9s
[CV 5/5; 20/20] START eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.1843781878273576, max_depth=14, n_estimators=1000


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[CV 5/5; 20/20] END eval_metric=<function mean_absolute_percentage_error at 0x0000022CD22251F0>, learning_rate=0.1843781878273576, max_depth=14, n_estimators=1000;, score=(train=-4056.465, test=-6679.028) total time=  11.7s


In [50]:
RandomSearch_results = pandas.DataFrame(RandomSearch.cv_results_)                 
print("Random Search: \tBest parameters: ", RandomSearch.best_params_, f", Best scores: {RandomSearch.best_score_:.3f}")

Random Search: 	Best parameters:  {'eval_metric': <function mean_absolute_percentage_error at 0x0000022CD22251F0>, 'learning_rate': 0.02244961377517401, 'max_depth': 5, 'n_estimators': 1000} , Best scores: -6434.880


In [70]:
model3 = XGBRegressor(max_depth = 8, eta = 0.15407279965399678, eval_metric = 'mape',n_estimators=1000)
model3.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=10)

[0]	validation_0-mape:0.84422
[1]	validation_0-mape:0.71594
[2]	validation_0-mape:0.60906
[3]	validation_0-mape:0.51979
[4]	validation_0-mape:0.44559
[5]	validation_0-mape:0.38322
[6]	validation_0-mape:0.33121
[7]	validation_0-mape:0.28754
[8]	validation_0-mape:0.25117
[9]	validation_0-mape:0.22082
[10]	validation_0-mape:0.19538
[11]	validation_0-mape:0.17415
[12]	validation_0-mape:0.15648
[13]	validation_0-mape:0.14176
[14]	validation_0-mape:0.12954
[15]	validation_0-mape:0.11940
[16]	validation_0-mape:0.11117
[17]	validation_0-mape:0.10448
[18]	validation_0-mape:0.09917
[19]	validation_0-mape:0.09485
[20]	validation_0-mape:0.09151
[21]	validation_0-mape:0.08892
[22]	validation_0-mape:0.08694
[23]	validation_0-mape:0.08539
[24]	validation_0-mape:0.08421
[25]	validation_0-mape:0.08333
[26]	validation_0-mape:0.08270
[27]	validation_0-mape:0.08216
[28]	validation_0-mape:0.08174
[29]	validation_0-mape:0.08144
[30]	validation_0-mape:0.08124
[31]	validation_0-mape:0.08102
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             eta=0.15407279965399678, eval_metric='mape', gamma=0, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.154072806, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [71]:
y_preds_train = model3.predict(X_val,iteration_range=(0,model.best_iteration))
print(y_preds_train[0:5])
print(y_val[0:5])

[138083.38  220708.66  138410.56  102004.14  114011.336]
18168     153298.265625
98202     230593.859375
98537     148630.921875
128116    109284.562500
156738    127021.703125
Name: p_truth_E, dtype: float64


In [72]:
y_pred_test = model3.predict(test[imp_var],iteration_range=(0,model.best_iteration))
print(y_pred_test)
print(y_pred_test.shape)

[ 79257.695  80450.516 222874.61  ... 115958.555  76996.06  149230.19 ]
(160651,)


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [73]:
with open('Regression_AndreasJørgensen_XGBRegressor.txt','w') as f:
    for i in range(len(y_pred_test)):
        f.write(str(i))
        f.write(',')
        f.write(str(y_pred_test[i]))
        f.write('\n')

In [74]:
with open('Regression_AndreasJørgensen_XGBRegressor_Variablelist.txt','w') as f:
    for i in range(len(imp_var)):
        f.write(str(imp_var[i]))
        f.write('\n')